####Load data

In [ ]:
X,y=pd.read_csv(dir+'X.csv'),pd.read_csv(dir+'y.csv')

/usr/local/lib/python3.10/dist-packages/openml/datasets/functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


This is dataset 'credit-g', the target feature is 'class'


In [ ]:
df=pd.concat([X,y],axis=1)

In [ ]:
df

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,checking_status,credit_history,purpose,...,employment,personal_status,other_parties,property_magnitude,other_payment_plans,housing,job,own_telephone,foreign_worker,class
0,18,3049.0,1,1,45,1,1,>=200,existing paid,furniture/equipment,...,<1,female div/dep/mar,none,life insurance,stores,own,unskilled resident,none,yes,good
1,24,1927.0,3,2,33,2,1,no checking,critical/other existing credit,education,...,1<=X<4,female div/dep/mar,none,car,none,own,skilled,yes,yes,good
2,12,640.0,4,2,49,1,1,no checking,existing paid,new car,...,1<=X<4,male div/sep,none,real estate,none,own,unskilled resident,none,yes,good
3,6,2063.0,4,3,30,1,1,0<=X<200,existing paid,radio/tv,...,<1,male mar/wid,none,car,none,rent,high qualif/self emp/mgmt,yes,yes,good
4,12,1526.0,4,4,66,2,1,<0,critical/other existing credit,used car,...,>=7,male single,none,no known property,none,for free,high qualif/self emp/mgmt,none,yes,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12,1295.0,3,1,25,1,1,0<=X<200,existing paid,new car,...,<1,female div/dep/mar,none,car,none,rent,skilled,none,yes,bad
996,15,802.0,4,3,37,1,2,0<=X<200,existing paid,radio/tv,...,>=7,male single,none,car,none,own,skilled,none,yes,bad
997,15,1275.0,4,2,24,1,1,<0,existing paid,domestic appliance,...,1<=X<4,female div/dep/mar,none,car,none,rent,skilled,none,yes,bad
998,18,1808.0,4,1,22,1,1,no checking,delayed previously,furniture/equipment,...,4<=X<7,female div/dep/mar,none,real estate,none,own,skilled,none,yes,bad


In [ ]:
print(f'Orginal dataset shape {df.shape}')

n_samples=df.shape[0]
limitPer = n_samples * 0.5
df = df.dropna(thresh=limitPer, axis=1)
print(f'Drop columns with high missing values {df.shape}')

to_drop=[]
for col in df.columns:
  col_unique_itmes = df[col].nunique()
  ##print single value features
  if (col_unique_itmes == 1):
      to_drop.append(col)
      print(col + " has been added to drop list due to there is single value in this feature")
  elif (col_unique_itmes == df.shape[0]):
      to_drop.append(col)
      print(col + " has been added to drop list as it may be id as values are totally unique")

df = df.drop(to_drop,axis=1)
print(f'Drop columns with single and totally unique values {df.shape}')

Orginal dataset shape (1000, 21)
Drop columns with high missing values (1000, 21)
Drop columns with single and totally unique values (1000, 21)


####Data profiling - schema detection

In [ ]:
def check_if_datetime(x):
    try:
        pd.to_datetime(x)
    except (RuntimeError, TypeError, NameError, IOError, ValueError):
        return False
    else:
        return True


# Define function
def check_if_numeric(x):
    try:
        pd.to_numeric(x)
    except (RuntimeError, TypeError, NameError, IOError, ValueError):
        return False
    else:
        return True


def identify_variable_type(x):
    # Is the column empty? If so, it will be classified as 'NA':
    if (x.dropna().empty == True):
        return "NA"

    if x.dropna().nunique() == len(x)*0.99:
        return "u_manual"

    # Is the variable categorical? We check the number of unique values:
    if ((x.dropna().shape[0] < 3000 and x.dropna().nunique() < 30) or (x.dropna().nunique() < 50 and x.dropna().shape[0] >= 3000)):
        if x.dropna().nunique() == 2:
            return "c_binary"
        else:
            if (check_if_numeric(x) == True):
                return "c_integer"
            else:
                return "c_categorical"
    # If no numbers are present, we classify it as a string:
    elif (x.astype(str).str.contains(r"[0-9]").any() == False):
        return "c_string"
    # We then check if it's numeric, or predominantly numeric with some exceptions:
    elif (check_if_numeric(x) == True):
        if (x.dropna() % 1 == 0).all():
            return "n_integer"
        else:
            return "n_float"
    # next, we check if it's a date or a time, or predominantly datetime with some exceptions:
    elif (check_if_datetime(x) == True):
        return "n_datetime"
    elif (x.astype(str)[x.astype(str).str.contains(r"[0-9]") == False].nunique() < 11 and
          check_if_datetime(x[x.astype(str).str.contains(r"[0-9]") == True]) == True):
        return "n_datetime"
    # If none of the above apply, we classify the variable as string:
    else:
        return "u_manual"


def remove_dtname(text,list_dt):
      for i in list_dt:
          text = text.replace(i, '')
      return text

def prepend(list, str):
    str += '{0}'
    list = [str.format(i) for i in list]
    return (list)


def datatype_detect(data,cat_enc=True):

  df=data.copy()

  column_types=df.apply(identify_variable_type)
  column_types = prepend(column_types, '_')
  column_types=[s[:6] for s in column_types]
  new_column_names = [i + j for i, j in zip(list(df.columns), column_types)]

  df.columns = new_column_names

  col_names=df.columns.to_list()

  c_col=[s for s in col_names if '_c_' in s]
  c_col_index=[df.columns.get_loc(c) for c in c_col]
  n_col=[s for s in col_names if '_n_' in s]
  int_col=[s for s in col_names if 'int' in s]

  lists=['_c_cat','_n_int','_n_flo','_c_bin','_c_str','_c_int']

  c_col=[remove_dtname(i,lists) for i in c_col]
  n_col=[remove_dtname(i,lists) for i in n_col]
  int_col=[remove_dtname(i,lists) for i in int_col]

  cat_dict={}

  if cat_enc:
    data,cat_dict=label_encoding(data,c_col,int_col)

  for col in data.columns:
    if col in int_col:
      data[col] = data[col].astype('int')
      # data[col]=pd.to_numeric(data[col], errors='coerce').fillna(0)
    else:
      data[col] = data[col].astype('float')

  return data,cat_dict,c_col,n_col,c_col_index,int_col

def label_encoding(data,c_col,int_col):

  real=data.copy()
  cat_dict={}
  for i in c_col:
    if i not in int_col:
      real[i]=real[i].astype('category')
      cat_dict[i] = dict(enumerate(real[i].cat.categories))
      real[i]=real[i].cat.codes
      real[i]=real[i].astype('int')

  return real,cat_dict

In [ ]:
data,cat_dict,c_col,n_col,c_col_index,int_col=datatype_detect(df)

In [ ]:
data.head()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents,checking_status,credit_history,purpose,...,employment,personal_status,other_parties,property_magnitude,other_payment_plans,housing,job,own_telephone,foreign_worker,class
0,18,3049,1,1,45,1,0.0,2.0,3.0,3.0,...,2.0,0.0,2.0,1.0,2.0,1.0,3.0,0.0,1.0,1.0
1,24,1927,3,2,33,2,0.0,3.0,1.0,2.0,...,0.0,0.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,12,640,4,2,49,1,0.0,3.0,3.0,4.0,...,0.0,1.0,2.0,3.0,1.0,1.0,3.0,0.0,1.0,1.0
3,6,2063,4,3,30,1,0.0,0.0,3.0,6.0,...,2.0,2.0,2.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0
4,12,1526,4,4,66,2,0.0,1.0,1.0,9.0,...,3.0,3.0,2.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
cat_dict

{'num_dependents': {0: 1, 1: 2},
 'checking_status': {0: '0<=X<200', 1: '<0', 2: '>=200', 3: 'no checking'},
 'credit_history': {0: 'all paid',
  1: 'critical/other existing credit',
  2: 'delayed previously',
  3: 'existing paid',
  4: 'no credits/all paid'},
 'purpose': {0: 'business',
  1: 'domestic appliance',
  2: 'education',
  3: 'furniture/equipment',
  4: 'new car',
  5: 'other',
  6: 'radio/tv',
  7: 'repairs',
  8: 'retraining',
  9: 'used car'},
 'savings_status': {0: '100<=X<500',
  1: '500<=X<1000',
  2: '<100',
  3: '>=1000',
  4: 'no known savings'},
 'employment': {0: '1<=X<4', 1: '4<=X<7', 2: '<1', 3: '>=7', 4: 'unemployed'},
 'personal_status': {0: 'female div/dep/mar',
  1: 'male div/sep',
  2: 'male mar/wid',
  3: 'male single'},
 'other_parties': {0: 'co applicant', 1: 'guarantor', 2: 'none'},
 'property_magnitude': {0: 'car',
  1: 'life insurance',
  2: 'no known property',
  3: 'real estate'},
 'other_payment_plans': {0: 'bank', 1: 'none', 2: 'stores'},
 'housin

####Statistical Analysis

In [ ]:
!pip install pyreadstat --quiet

import pandas as pd
import numpy as np

import pickle
!pip install researchpy --quiet
import researchpy as rp
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.2 MB/s eta 0:00:00


In [ ]:
rp.summarize(df[n_col])

,Name,N,Mean,Median,Variance,SD,SE,95% Conf. Interval
0,duration,1000,20.903,18.0,145.415,12.0588,0.3813,duration
1,credit_amount,1000,3271.258,2319.5,7967843.4709,2822.7369,89.2628,credit_amount
2,age,1000,35.546,33.0,129.4013,11.3755,0.3597,age


In [ ]:
rp.summary_cat(df[c_col])

/usr/local/lib/python3.10/dist-packages/researchpy/summary.py:225: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for ix, df_col in group1.iteritems():


,Variable,Outcome,Count,Percent
0,installment_commitment,4,476,47.6
1,,2,231,23.1
2,,3,157,15.7
3,,1,136,13.6
4,residence_since,4,413,41.3
...,...,...,...,...
65,,yes,404,40.4
66,foreign_worker,yes,963,96.3
67,,no,37,3.7
68,class,good,700,70.0


In [ ]:
y_name="class"
ss,rs={},{}
for i in n_col:
  summary, results = rp.ttest(group1= df[i][df[y_name] == df[y_name].unique()[1]], group1_name= "Postive",
                              group2= df[i][df[y_name] == df[y_name].unique()[0]], group2_name= "Negative")
  ss[i]=summary
  rs[i]=results


pd.concat(ss)

/usr/local/lib/python3.10/dist-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/usr/local/lib/python3.10/dist-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/usr/local/lib/python3.10/dist-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


Variable       N         Mean           SD          SE  \
duration      0   Postive   300.0    24.860000    13.282639    0.766874   
              1  Negative   700.0    19.207143    11.079564    0.418768   
              2  combined  1000.0    20.903000    12.058814    0.381333   
credit_amount 0   Postive   300.0  3938.126667  3535.818955  204.140603   
              1  Negative   700.0  2985.457143  2401.472278   90.767120   
              2  combined  1000.0  3271.258000  2822.736876   89.262778   
age           0   Postive   300.0    33.963333    11.222379    0.647924   
              1  Negative   700.0    36.224286    11.381145    0.430167   
              2  combined  1000.0    35.546000    11.375469    0.359724   

                   95% Conf.     Interval  
duration      0    23.350847    26.369153  
              1    18.384949    20.029337  
              2    20.154694    21.651306  
credit_amount 0  3536.392318  4339.861015  
              1  2807.248285  3163.666001  
              2  3096.093951  3446.422049  
age           0    32.688264    35.238403  
              1    35.379712    37.068860  
              2    34.840099    36.251901

In [ ]:
cb={}
for i in c_col:
  cb[i]=pd.concat([pd.crosstab(df[i],df[y_name]),pd.crosstab(df[i],df[y_name]).apply(lambda r: r/r.sum(), axis=0)],axis=1)

pd.concat(cb)

class                        bad  good       bad      good
installment_commitment 1      34   102  0.113333  0.145714
                       2      62   169  0.206667  0.241429
                       3      45   112  0.150000  0.160000
                       4     159   317  0.530000  0.452857
residence_since        1      36    94  0.120000  0.134286
...                          ...   ...       ...       ...
own_telephone          yes   113   291  0.376667  0.415714
foreign_worker         no      4    33  0.013333  0.047143
                       yes   296   667  0.986667  0.952857
class                  bad   300     0  1.000000  0.000000
                       good    0   700  0.000000  1.000000

[70 rows x 4 columns]

####Split and save data into 5 seeds

In [ ]:
from sklearn.model_selection import train_test_split

for i in range(5):
  X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=i)
  baseline=pd.concat([X_train,y_train],axis=1)
  df_test=pd.concat([X_test,y_test],axis=1)
  baseline.to_csv(dir+'synthetic/seed' + str(i) +'/baseline.csv',index=False)
  df_test.to_csv(dir+'synthetic/seed' + str(i) +'/df_test.csv',index=False)